<a href="https://colab.research.google.com/github/KevinFallon/fastai/blob/main/FastAI_Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *
path = untar_data(URLs.PETS)

     |████████████████████████████████| 719 kB 1.7 MB/s 
     |████████████████████████████████| 451 kB 11.0 MB/s 
     |████████████████████████████████| 1.3 MB 45.9 MB/s 
     |████████████████████████████████| 5.5 MB 12.6 MB/s 
     |████████████████████████████████| 1.6 MB 52.9 MB/s 
     |████████████████████████████████| 212 kB 58.1 MB/s 
     |████████████████████████████████| 182 kB 57.1 MB/s 
     |████████████████████████████████| 115 kB 61.1 MB/s 
     |████████████████████████████████| 127 kB 55.0 MB/s 
     |████████████████████████████████| 7.6 MB 55.2 MB/s 
Mounted at /content/gdrive


In [ ]:
Path.BASE_PATH = path
path.ls()

In [ ]:
(path/"images").ls()

In [ ]:
fname = (path/"images").ls()[0]
print(fname)
re.findall(r'(.+)_\d+.jpg$', fname.name)

In [ ]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 # The two lines after this are for "Presizing"
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))

'''
Presizing is a way to do image augmentation that is designed to minimize data
destruction while maintaining good performance. We need images to have the same
dimensions so they can collate into tensors to be passed to the GPU. Additionally,
we want to minimize the number of distinct augmentation computations we pefrom. 


transform images into uniform sizes for more efficient processing on the GPU.

Presizing strategies:
1. Resize images to relatively "large" dimensions. Dimenions significantly larger
than the target training dimensions.
2. Compose all the common augmentation operations into one and perform combined
operation on the GPU only at the end of processing, rather than performing operations individually
and interpolating multiple times.
'''
dls = pets.dataloaders(path/'images')

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '0302a1f901a545f2986fcdd53310b214')
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('contentUrl')
# print(ims)

from fastdownload import download_url
dest = Path.cwd()/'images'/'grizzly.jpg'
download_url(ims[0], dest, show_progress=False)
im = Image.open(dest)
im.to_thumb(256, 256)

In [ ]:
dblock1 = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                    get_y=parent_label,
                    item_tfms=Resize(460),
)

dls1 = dblock1.dataloaders([(Path.cwd()/'images'/'grizzly.jpg')]*100, bs=4)
dls1.train.get_idxs = lambda: Inf.ones
# x is independent variable, so a TensorImage
# y is dependent variable, so a 1 rank tensor of size `bs` representing possible
# categories.
x,y = dls1.valid.one_batch()
_, axs = subplots(1, 2)

x1 = TensorImage(x.clone())
print("before")
print(x1.size())
x1 = x1.affine_coord(sz=224)
print("after")
print(x1.size())
x1 = x1.rotate(draw=30, p=1.)
x1 = x1.zoom(draw=1.2, p=1.)
x1 = x1.warp(draw_x=-0.2, draw_y=0.2, p=1.0)

tfms = setup_aug_tfms([Rotate(draw=30, p=1, size=224), Zoom(draw=1.2, p=1., size=224),
                       Warp(draw_x=-0.2, draw_y=0.2, p=1., size=224)])
pipeline = Pipeline(tfms)
x = pipeline(x)
TensorImage(x[0]).show(ctx=axs[0])
TensorImage(x1[0]).show(ctx=axs[1]);
'''
It has been found thatin practice using presizing significantly improves the
accuracy of models, and often results in speedups too.
'''

In [ ]:
'''
its easy to make unnoticed mistakes while setting up the datablock so its important
to verify the labeled data appears to be correct.
'''
dls.show_batch(nrows=1, ncols=3)

In [ ]:
# this is an example of which we forget to set the resize function. its useful
# to use the 'summary' method to check for mistakes.
pets1 = DataBlock(blocks = (ImageBlock, CategoryBlock),
                  get_items=get_image_files,
                  splitter=RandomSplitter(seed=42),
                  get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'))
# purposely fails with an error describing non-consistent sizing of images
# pets1.summary(path/'images')

In [ ]:
'''
it's important to train any model asap so we can some insight on how to best proceed and iterate
is the data good enough? maybe we already have a good enough model and dont need to spend
additional engineering time on new features
'''
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

In [ ]:
'''
cross-entropy loss is what the fastai library uses for image data as input
with a categorical output. fastai tries its best to choose an appropriate loss
function when its not supplied.
cross-entropy loss function has 2 benefits:
1. it works even when our dependent variable has more than two categories
2. it results in faster and more reliable training.


'''

x, y = dls.one_batch()